<img src="images/openclass-192.jpeg" />


<div class="bg-primary text-center">
    - Summary -
</div>

Warehouse automation with robots is crucial in the present for enhancing operational efficiency, reducing labor costs, and ensuring accuracy in inventory management. In the future, it will be even more vital as demand for faster fulfillment and scalability increases, driven by the growth of e-commerce and the need for resilient supply chains.

In this Open Class, you will learn how to **combine Perception and Grasping techniques** to explore warehouse automation with ROS 2.

What you'll learn:
- Combining **Perception** and **Grasping** for Automation: Develop skills to seamlessly integrate perception and grasping techniques to create advanced robotic solutions that can autonomously detect, pick, and place objects in a warehouse environment.
 
You'll be using the **BOTBOX** throughout the training.

## BOTBOX is a lab-in-a-box to teach robotics, including off-the-shelf robots, the environment, simulations, and projects for your students.

### Your students need to install nothing in order to start programming the robots. Everything is web based and works in any computer.

Have full control of your student’s progress.

![](images/IMG_3494.JPG)

![](images/demosim.png)

## The Botbox package includes:

![](images/botbox_package.png)

## Get more info at https://www.theconstruct.ai/botbox-warehouse-lab/

<div class="bg-primary text-center">
    - End of Summary -
</div>

<div>
    <h1 class="text-center">
        <span class="">Launch the simulation</span>
    </h1>
</div>

To launch the project simulation:

1. Open a terminal by clicking:

<img src="images/rosject_toolbar_terminal.png"/>

2. Copy following command in terminal:

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in a Terminal
</span>

In [ ]:
ros2 launch tortoisebot_bringup simulation.launch.py

**Wait around 30 seconds** for simulation to start. It should automatically appear in a Gazebo window.

If it doesn't automatically appear, open the Gazebo window by clicking:

<img src="images/rosject_toolbar_gazebo.png"/>

Gazebo window should show the Tortoisebot Warehouse world:

<img src="images/sim_gripper.png"/>

<div class="bg-primary text-center">
    - Notes -
</div>

* The terminal where the simulation was launched is occupied, it cannot be used unless you simulation is killed. Open new terminals to run other processes.
* The simulation or the real robot is needed. If none of these are running, then there will be no incoming data and ROS 2 nodes will not work.

<div class="bg-primary text-center">
    - End Notes -
</div>

<div>
    <h1 class="text-center">
        <span class="text-primary">Important Information</span>
        &nbsp;
        <span class="">Grasping System</span>
    </h1>
</div>

You might have noticed the simplified gripper in the simulation. In the real robot, make sure that the gripper is also attached:

<img src="images/humblegrip.jpg" width="200" align="center"/>

If the griper is not attached, use the provided screw, washers and nut to attach the gripper bracket to the bottom of the robot:

<img src="images/humblegrip_screw.jpg" width="275" align="left"/>
<img src="images/humblegrip_bottom.jpg" width="275" align="right"/>

### Operate Gripper

To **close** gripper:

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in a Terminal
</span>

In [ ]:
ros2 action send_goal /gripper_controller/gripper_cmd control_msgs/action/GripperCommand "command:
  position: -0.02
  max_effort: 0.0"

To **open** gripper:

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in a Terminal
</span>

In [ ]:
ros2 action send_goal /gripper_controller/gripper_cmd control_msgs/action/GripperCommand "command:
  position: 0.02
  max_effort: 0.0"

You'll notice the `action` prefix to the command, this will come in important later.

<div>
    <h1 class="text-center">
        <span class="text-primary">Today's Mission</span>
        &nbsp;
        <span class="">Transport the goods 🚚</span>
    </h1>
</div>

You'll notice our warehouse looks a little different from last time. This is because the factory has been busy producing goods, and the factory floor has gone through a bit of an upgrade. 

<img src="images/map.png" style="width: 400px" />

We will be starting at the same place as the previous classes, which will be within the process station. This is where the factory produces the product which in this case is these cubes with interesting markers on them (if you did the previous few open classes, you will already know what these are). 

<img src="images/map_zoomed.png" style="width: 200px" />

For this class, your mission is fairly simple: pick up our product from the processing station and return to the factory line. 

<div>
    <h1 class="text-center">
        <span class="text-primary">Task 1</span>
        &nbsp;
        <span class="">Pallet cube perception</span>
    </h1>
</div>

As with all of our previous lessons, let's first take a look at our product and see what we have to work with.

Let's create a new package for this.

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in a Terminal
</span>

In [ ]:
cd ~/ros2_ws/src/

In [ ]:
ros2 pkg create --build-type ament_python pick_and_place --dependencies rclpy std_msgs geometry_msgs cv_bridge sensor_msgs

Now let's create our script within the package to run. 

![](images/file_structure.png)

<span class="badge badge-pill badge-primary">
    <i class="fa fa-file"></i>
    &nbsp;
    pick_and_place.py
</span>

In [ ]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2

class ImageSubscriber(Node):
    def __init__(self):
        super().__init__('pick_and_place')
        self.subscription = self.create_subscription(
            Image,
            '/camera/image_raw',
            self.listener_callback,
            10)
        self.subscription  # prevent unused variable warning

        self.bridge = CvBridge()

    def listener_callback(self, msg):
        self.get_logger().info('Receiving video frame')
        # Convert ROS Image message to OpenCV image
        cv_image = self.bridge.imgmsg_to_cv2(msg, 'bgr8')
        
        # Display image
        cv2.imshow("Camera Feed", cv_image)
        cv2.waitKey(1)

def main(args=None):
    rclpy.init(args=args)
    image_subscriber = ImageSubscriber()

    try:
        rclpy.spin(image_subscriber)
    except KeyboardInterrupt:
        pass

    # Destroy the node explicitly
    image_subscriber.destroy_node()
    rclpy.shutdown()

    # Close any OpenCV windows
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()

Now we will need to modify the `setup.py` file in order for ROS to see our new file as a node it can run. 

<span class="badge badge-pill badge-primary">
    <i class="fa fa-file"></i>
    &nbsp;
    setup.py
</span>

In [ ]:
from setuptools import setup

package_name = 'pick_and_place'

setup(
    name=package_name,
    version='0.0.0',
    packages=[package_name],
    data_files=[
        ('share/ament_index/resource_index/packages',
            ['resource/' + package_name]),
        ('share/' + package_name, ['package.xml']),
    ],
    install_requires=['setuptools'],
    zip_safe=True,
    maintainer='user',
    maintainer_email='user@todo.todo',
    description='TODO: Package description',
    license='TODO: License declaration',
    tests_require=['pytest'],
    entry_points={
        'console_scripts': [
            'pick_and_place = pick_and_place.pick_and_place:main'
        ],
    },
)

Now lets build and run our new script. 

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in a Terminal
</span>

In [ ]:
cd ~/ros2_ws

# Build the packages in the ros workspace
colcon build 

# Source our setup varibles so ros knows where all the built files are
source install/setup.bash

# Run our new node
ros2 run pick_and_place pick_and_place

If everything is correct you should see the following open up in graphical tools.

![](images/task1.png)

We see a brief glimpse of one of the pallet cubes. 

If you look a little closely ... what's that? 

For the people that attended "Perception with AprilTags for ROS2" may already know what this is.

These are Aruco tags.

<img src="https://docs.opencv.org/4.x/markers.jpg" style="width: 200px" />

ArUco markers are 2D binary-encoded fiducial patterns designed to be quickly located by computer vision systems. By performing a set of calculations using the patterns detected by the camera, compute the 3D pose of the tag relative to the position of the robot.

As you'd imagine, it's extremely important for us to know the position of the tag precisely, as we need to align ourselves with the pallet for our gripper to be able to grab it. 

Now for this class, we will be using the package `ros2_aruco` to get the detected tag id and it's pose. 

* `ros2_aruco` [public repo](https://github.com/JMU-ROBOTICS-VIVA/ros2_aruco) from James Madison University Robotics. This was also used to generate the aruco markers in the WarehousePlanet.

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in a Terminal
</span>

In [ ]:
ros2 run ros2_aruco aruco_node marker_size:=0.05

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in a Terminal
</span>

In [ ]:
ros2 run teleop_twist_keyboard teleop_twist_keyboard

If a pallet cube is in view of the robot camera, then you should see a published topic `/aruco_markers`. This contains an array of all marker poses (with respect to robot camera) and its corresponding marker ID:

In [ ]:
int64[] marker_ids
geometry_msgs/Pose[] poses

Since we want to run the entire program in one command and not split across terminals we can make a single launch files that runs both nodes from our own package and the `ros2_aruco` package.

<span class="badge badge-pill badge-primary">
    <i class="fa fa-file"></i>
    &nbsp;
    launch/pick_and_place.launch.py
</span>

In [ ]:
from launch import LaunchDescription
from launch_ros.actions import Node

def generate_launch_description():
    return LaunchDescription([
        Node(
            package='pick_and_place',
            executable='pick_and_place',
            name='pick_and_place_node',
            output='screen',
        ),
        Node(
            package='ros2_aruco',
            executable='aruco_node',
            name='aruco_node',
            output='screen',
            parameters=[{'marker_size': 0.05}]
        )
    ])

if __name__ == '__main__':
    generate_launch_description()

<span class="badge badge-pill badge-primary">
    <i class="fa fa-file"></i>
    &nbsp;
    setup.py
</span>

In [ ]:
from setuptools import setup
from glob import glob 
import os

package_name = 'pick_and_place'

setup(
    name=package_name,
    version='0.0.0',
    packages=[package_name],
    data_files=[
        ('share/ament_index/resource_index/packages',
            ['resource/' + package_name]),
        ('share/' + package_name, ['package.xml']),
        (os.path.join('share', package_name), glob('launch/*.launch.py'))
    ],
    install_requires=['setuptools'],
    zip_safe=True,
    maintainer='user',
    maintainer_email='user@todo.todo',
    description='TODO: Package description',
    license='TODO: License declaration',
    tests_require=['pytest'],
    entry_points={
        'console_scripts': [
            'pick_and_place = pick_and_place.pick_and_place:main'
        ],
    },
)

Now lets build and run our new script. 

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in a Terminal
</span>

In [ ]:
cd ~/ros2_ws

# Build the packages in the ros workspace
colcon build 

# Source our setup varibles so ros knows where all the built files are
source install/setup.bash

# Run our new node
ros2 launch pick_and_place pick_and_place.launch.py

Now you'll be able to see that once the tag enters the camera frame it gets detected.

![](images/task-1-1.gif)

Now that we can see we are getting the information from the tag lets display it on our image view in our script.

For that we will need to subscribe to the `aruco_markers` topic that will have all of the information we need and then display it using opencv. 

Lets decide on what we are going to do.

1. Subscribe to the `aruco_markers` topic.
2. Add a callback that will take the pose information that we need and store them into the class. 
3. Using opencv display the tag id, the position using the display text function 
4. Draw a circle around the center of the detected tag
5. Draw a line from our current position to the center of the tag 
6. Determine the angle from our current position to the tag using the orientation information from the tag

Since the `aruco_markers` topic publishes a custom message we will have to import that message definition first.  

```python
from ros2_aruco_interfaces.msg import ArucoMarkers
```

Now we can add our subscriber 

```python
        self.marker_subscription = self.create_subscription(
            ArucoMarkers,
            '/aruco_markers',
            self.marker_callback,
            10)
```

And have a variable that stores markers detected

```python
        self.markers = []
```

You’ll notice that we defined a callback when we added our subscriber

```python
    def marker_callback(self, msg):
        self.get_logger().info('Receiving marker information')
        self.markers = []
        for i, marker_id in enumerate(msg.marker_ids):
            position = msg.poses[i].position
            orientation = msg.poses[i].orientation
            self.markers.append({
                'id': marker_id,
                'position': {
                    'x': position.x,
                    'y': position.y,
                    'z': position.z
                },
                'orientation': {
                    'x': orientation.x,
                    'y': orientation.y,
                    'z': orientation.z,
                    'w': orientation.w
                }
            })
```

This callback stores information about the position and orientation of the detected pose to the markers array. 

Now that we have the markers array being populated lets display it in our opencv frame.

```python
        # Center bottom of the frame
        center_bottom = (cv_image.shape[1] // 2, cv_image.shape[0])

        for marker in self.markers:
            marker_id = marker['id']
            position = marker['position']
            orientation = marker['orientation']
            
            # Convert position to pixel coordinates
            pixel_x = int(position['x'] * 1000 + cv_image.shape[1] / 2)
            pixel_y = int(position['y'] * 1000 + cv_image.shape[0] / 2)

            # Draw circle at marker position
            cv2.circle(cv_image, (pixel_x, pixel_y), 10, (0, 0, 255), -1)

            # Draw line from center bottom to marker position
            cv2.line(cv_image, center_bottom, (pixel_x, pixel_y), (0, 255, 0), 2)

            # Calculate the yaw angle in degrees from quaternion orientation
            quaternion = (orientation['x'], orientation['y'], orientation['z'], orientation['w'])
            euler = tf_transformations.euler_from_quaternion(quaternion)
            yaw_deg = np.degrees(euler[2]) * 100  # Extract yaw (z-axis rotation) and convert to degrees

            # Display position and yaw angle
            position_text = f"ID: {marker_id} Pos: ({position['x']:.2f}, {position['y']:.2f}, {position['z']:.2f})"
            yaw_text = f"Yaw: {yaw_deg:.2f} degrees"
            cv2.putText(cv_image, position_text, (10, 50 + 20 * marker_id), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(cv_image, yaw_text, (10, 70 + 20 * marker_id), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
```

In this for each and every marker that is detected which in this case is going to be only one marker we are going to calculated the pixel coordinates of the marker position and then draw a circle so we can see where the tag is being detected as.

Then we draw a line from the center bottom to the marker center, this will help us see our position relative to the marker position.

After that we calculate the angle difference using the orientation information by first changing the quaternion to euler coordinates and then to degrees. 

And finally we add the text with the ID, Pos and Yaw.

<span class="badge badge-pill badge-primary">
    <i class="fa fa-file"></i>
    &nbsp;
    pick_and_place.py
</span>

In [ ]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Image
from ros2_aruco_interfaces.msg import ArucoMarkers
from cv_bridge import CvBridge
import cv2
import numpy as np
import tf_transformations

class ImageSubscriber(Node):
    def __init__(self):
        super().__init__('pick_and_place')
        self.image_subscription = self.create_subscription(
            Image,
            '/camera/image_raw',
            self.image_callback,
            10)
        self.marker_subscription = self.create_subscription(
            ArucoMarkers,
            '/aruco_markers',
            self.marker_callback,
            10)
        
        self.bridge = CvBridge()
        self.markers = []

    def image_callback(self, msg):
        self.get_logger().info('Receiving video frame')
        cv_image = self.bridge.imgmsg_to_cv2(msg, 'bgr8')

        # Center bottom of the frame
        center_bottom = (cv_image.shape[1] // 2, cv_image.shape[0])

        for marker in self.markers:
            marker_id = marker['id']
            position = marker['position']
            orientation = marker['orientation']
            
            # Convert position to pixel coordinates
            pixel_x = int(position['x'] * 1000 + cv_image.shape[1] / 2)
            pixel_y = int(position['y'] * 1000 + cv_image.shape[0] / 2)

            # Draw circle at marker position
            cv2.circle(cv_image, (pixel_x, pixel_y), 10, (0, 0, 255), -1)

            # Draw line from center bottom to marker position
            cv2.line(cv_image, center_bottom, (pixel_x, pixel_y), (0, 255, 0), 2)

            # Calculate the yaw angle in degrees from quaternion orientation
            quaternion = (orientation['x'], orientation['y'], orientation['z'], orientation['w'])
            euler = tf_transformations.euler_from_quaternion(quaternion)
            yaw_deg = np.degrees(euler[2]) * 100  # Extract yaw (z-axis rotation) and convert to degrees

            # Display position and yaw angle
            position_text = f"ID: {marker_id} Pos: ({position['x']:.2f}, {position['y']:.2f}, {position['z']:.2f})"
            yaw_text = f"Yaw: {yaw_deg:.2f} degrees"
            cv2.putText(cv_image, position_text, (10, 50 + 20 * marker_id), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(cv_image, yaw_text, (10, 70 + 20 * marker_id), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        cv2.imshow("Camera Feed", cv_image)
        cv2.waitKey(1)

    def marker_callback(self, msg):
        self.get_logger().info('Receiving marker information')
        self.markers = []
        for i, marker_id in enumerate(msg.marker_ids):
            position = msg.poses[i].position
            orientation = msg.poses[i].orientation
            self.markers.append({
                'id': marker_id,
                'position': {
                    'x': position.x,
                    'y': position.y,
                    'z': position.z
                },
                'orientation': {
                    'x': orientation.x,
                    'y': orientation.y,
                    'z': orientation.z,
                    'w': orientation.w
                }
            })

def main(args=None):
    rclpy.init(args=args)
    image_subscriber = ImageSubscriber()

    try:
        rclpy.spin(image_subscriber)
    except KeyboardInterrupt:
        pass

    image_subscriber.destroy_node()
    rclpy.shutdown()

    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


You should see:

![](images/task-1-2.gif)

<div>
    <h1 class="text-center">
        <span class="text-primary">2</span>
        &nbsp;
        <span class="">Pallet cube grasping</span>
    </h1>
</div>

Now that we are able to see the cube we need to be able to grab the cube so that we can take it wherever we need to take it. 

For that we need to do a couple of steps.

1. Setup `cmd_vel` publisher. 
2. Turn until the tag is visible.
3. Open our gripper.
4. Approach the tag until we are sure that we have the cube inside of our gripper.
5. Close the gripper and stop the approach.

The turning is fairly simple so lets about the gripper for a bit.

You’ll have noticed the command for the gripper is a bit different that what you might be used to. 

```
ros2 action send_goal /gripper_controller/gripper_cmd control_msgs/action/GripperCommand "command: position: -0.02\n  max_effort: 0.0"
```

This is because `/gripper_controller/gripper_cmd` is actually an action server.

![](https://docs.ros.org/en/foxy/_images/Action-SingleActionClient.gif)

Now don’t worry we won’t be going much into action servers in this class but if your curious you can watch this video. 

https://www.youtube.com/watch?v=ICpsNT3lhaU



Lets first add our new publisher to `cmd_vel` and our action client for `/gripper_controller/gripper_cmd`

```python
    self.cmd_vel_publisher = self.create_publisher(Twist, '/cmd_vel', 10)
    self.gripper_action_client = ActionClient(self, GripperCommand, '/gripper_controller/gripper_cmd')
```

We will also add a new flags to track our gripper state.

```python
    self.gripper_opened = False
    self.gripper_closed = False
```

For our first step which is to turn left its quite simple, we just need to have a check that if there are no detected tags we turn left. 

```python
    twist = Twist()

    if not self.markers:
        # If no markers are detected, turn left
        twist.angular.z = 0.5  # Turn left at a constant rate
    else:
		    ...
		    
        for marker in self.markers:
            marker_id = marker['id']
            position = marker['position']
            orientation = marker['orientation']

						...

     # Publish the accumulated twist command
     self.cmd_vel_publisher.publish(twist)
```

Now once we have detected the tag we need to open our gripper. 

For that we will use,

```python
        # If marker detected
        if not self.gripper_opened:
            # Open the gripper
            self.send_gripper_command(0.02, 0.0)
            self.gripper_opened = True
```

This checks if our gripper is opened when its supposed to be the if its not we will use the `send_gripper_command` to open the gripper.

`send_gripper_command` is a new function that we will write that makes use of that action server client. This does essentially the asynchronous version of the command you ran earlier. 

```python
def send_gripper_command(self, position, max_effort):
    goal_msg = GripperCommand.Goal()
    goal_msg.command.position = position
    goal_msg.command.max_effort = max_effort

    self.gripper_action_client.wait_for_server()
    self.gripper_action_client.send_goal_async(goal_msg)
```

Ok now we have the gripper open and our cube in sight.

How are we supposed to approach the cube and make sure we are approaching it straight. For this we will be applying a simple proportional algorithm based off the offset of the robot from the center of the tag.

$C_z = error_x \times K_p$

Where:

- $C_z$ is the control output
- $error_x$ is the difference between the current position and the desired position (center of the tag)
- $K_p$ is the proportional gain

```python
                # Calculate the horizontal offset from the center
                offset = pixel_x - center_bottom[0]

                # Align with the tag using proportional control
                k_p = 0.007  # Proportional control constant
                twist.angular.z = -k_p * offset  # Proportional control for alignment

                # Move forward slowly
                twist.linear.x = 0.02
```

In [ ]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Image
from ros2_aruco_interfaces.msg import ArucoMarkers
from geometry_msgs.msg import Twist
from cv_bridge import CvBridge
from rclpy.action import ActionClient
from control_msgs.action import GripperCommand
import cv2
import numpy as np
import tf_transformations

class ImageSubscriber(Node):
    def __init__(self):
        super().__init__('pick_and_place')
        self.image_subscription = self.create_subscription(
            Image,
            '/camera/image_raw',
            self.image_callback,
            10)
        self.marker_subscription = self.create_subscription(
            ArucoMarkers,
            '/aruco_markers',
            self.marker_callback,
            10)
        
        self.cmd_vel_publisher = self.create_publisher(Twist, '/cmd_vel', 10)
        self.gripper_action_client = ActionClient(self, GripperCommand, '/gripper_controller/gripper_cmd')
        
        self.bridge = CvBridge()
        self.markers = []
        self.gripper_opened = False
        self.gripper_closed = False

    def send_gripper_command(self, position, max_effort):
        goal_msg = GripperCommand.Goal()
        goal_msg.command.position = position
        goal_msg.command.max_effort = max_effort

        self.gripper_action_client.wait_for_server()
        self.gripper_action_client.send_goal_async(goal_msg)

    def image_callback(self, msg):
        cv_image = self.bridge.imgmsg_to_cv2(msg, 'bgr8')

        # Center bottom of the frame
        center_bottom = (cv_image.shape[1] // 2, cv_image.shape[0])
        twist = Twist()

        if not self.markers:
            if not self.gripper_closed:
                # If no markers are detected, turn left
                twist.angular.z = 0.5  # Turn left at a constant rate
            else:
                twist.linear.x = 0.0
                twist.angular.z = 0.0
        else:
            # If marker detected
            if not self.gripper_opened:
                # Open the gripper
                self.send_gripper_command(0.02, 0.0)
                self.gripper_opened = True

            for marker in self.markers:
                marker_id = marker['id']
                position = marker['position']
                orientation = marker['orientation']
                
                # Convert position to pixel coordinates
                pixel_x = int(position['x'] * 1000 + cv_image.shape[1] / 2)
                pixel_y = int(position['y'] * 1000 + cv_image.shape[0] / 2)

                # Draw circle at marker position
                cv2.circle(cv_image, (pixel_x, pixel_y), 10, (0, 0, 255), -1)

                # Draw line from center bottom to marker position
                cv2.line(cv_image, center_bottom, (pixel_x, pixel_y), (0, 255, 0), 2)

                # Calculate the yaw angle in degrees from quaternion orientation
                quaternion = (orientation['x'], orientation['y'], orientation['z'], orientation['w'])
                euler = tf_transformations.euler_from_quaternion(quaternion)
                yaw_deg = np.degrees(euler[2]) * 100  # Extract yaw (z-axis rotation) and convert to degrees

                # Display position and yaw angle
                position_text = f"ID: {marker_id} Pos: ({position['x']:.2f}, {position['y']:.2f}, {position['z']:.2f})"
                yaw_text = f"Yaw: {yaw_deg:.2f} degrees"
                cv2.putText(cv_image, position_text, (10, 50 + 20 * marker_id), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                cv2.putText(cv_image, yaw_text, (10, 70 + 20 * marker_id), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # Calculate the horizontal offset from the center
                offset = pixel_x - center_bottom[0]

                # Align with the tag using proportional control
                k_p = 0.001  # Proportional control constant
                twist.angular.z = -k_p * offset  # Proportional control for alignment

                # Move forward slowly
                twist.linear.x = 0.02

                # Check if the marker is no longer visible (simulate the end of the task)
                if not self.markers:
                    twist.linear.x = 0.0
                    twist.angular.z = 0.0
                    if not self.gripper_closed:
                        self.send_gripper_command(-0.02, 0.0)
                        self.gripper_closed = True

        # Publish the accumulated twist command
        self.cmd_vel_publisher.publish(twist)

        cv2.imshow("Camera Feed", cv_image)
        cv2.waitKey(1)

    def marker_callback(self, msg):
        self.markers = []
        for i, marker_id in enumerate(msg.marker_ids):
            position = msg.poses[i].position
            orientation = msg.poses[i].orientation
            self.markers.append({
                'id': marker_id,
                'position': {
                    'x': position.x,
                    'y': position.y,
                    'z': position.z
                },
                'orientation': {
                    'x': orientation.x,
                    'y': orientation.y,
                    'z': orientation.z,
                    'w': orientation.w
                }
            })

def main(args=None):
    rclpy.init(args=args)
    image_subscriber = ImageSubscriber()

    try:
        rclpy.spin(image_subscriber)
    except KeyboardInterrupt:
        pass

    image_subscriber.destroy_node()
    rclpy.shutdown()

    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


![](images/final-1.gif)

<img src="images/humblegrip_grasp.jpg"/>

<div>
    <h1 class="text-center">
        <span class="text-primary">Challenge</span>
        &nbsp;
        <span class="">Autonomous Return and Release</span>
    </h1>
</div>

Once you've successfully grabbed the cube, your next challenge is to navigate back to the centerline and then release the gripper.

Here's what you need to do:

1. Program your robot to reverse its path and return to the centerline.
2. Once the robot has reached the centerline, program it to release the gripper and drop the cube.

Remember to use make sure of the gripper functions that we’ve used thus far.

<div>
    <h1 class="text-center">
        <span class="text-primary">Homework</span>
        &nbsp;
        <span class="">Combine !!!</span>
    </h1>
</div>


Now that you've understood how to:

1. Detect and track a cube using OpenCV and ArUco markers,
2. Open and close the gripper, and
3. Navigate using the line following skills

You'll have noticed a single cube in the storage bay. Your task, should you choose to accept it, is to use the line following skills you have learned in the previous open class to navigate to the storage bay, and using the information that you have learned in this open class, grab the cube. Once you have the cube, navigate to the dispatch area and release the cube.